In [1]:
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
movies = pd.read_csv('csv/movies.csv')
ratings = pd.read_csv('csv/ratings.csv')
credits = pd.read_csv('csv/credits.csv')

In [3]:
movies.duplicated().sum()

np.int64(0)

In [4]:
ratings.duplicated().sum()

np.int64(1511)

In [5]:
ratings = ratings.drop_duplicates(subset=["user_id", "movie_id"], keep="first")


In [6]:
ratings.duplicated().sum()

np.int64(0)

In [7]:
print(movies.shape)
print(ratings.shape)
print(credits.shape)

(4803, 20)
(10363, 3)
(4803, 4)


In [8]:
movies

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures

##Popularity Based

In [9]:
ratings_with_name = ratings.merge(movies, on="movie_id")

In [10]:
ratings_with_name.head(5)

,user_id,movie_id,rating,budget,genres,homepage,keywords,original_language,original_title,overview,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,1,99861,4.0,280000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/movies/movie/193/avengers_ag...,"[{""id"": 8828, ""name"": ""marvel comic""}, {""id"": ...",en,Avengers: Age of Ultron,When Tony Stark tries to jumpstart a dormant p...,...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-04-22,1405403694,141.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,A New Age Has Come.,Avengers: Age of Ultron,7.3,6767
1,626,44833,3.5,209000000,"[{""id"": 53, ""name"": ""Thriller""}, {""id"": 28, ""n...",NaN,"[{""id"": 1721, ""name"": ""fight""}, {""id"": 4410, ""...",en,Battleship,"When mankind beams a radio signal into space, ...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-04-11,303025485,131.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,The Battle for Earth Begins at Sea,Battleship,5.5,2114
2,1,150540,3.5,175000000,"[{""id"": 18, ""name"": ""Drama""}, {""id"": 35, ""name...",http://movies.disney.com/inside-out,"[{""id"": 1566, ""name"": ""dream""}, {""id"": 6513, ""...",en,Inside Out,"Growing up can be a bumpy road, and it's no ex...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2015-06-09,857611174,94.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Meet the little voices inside your head.,Inside Out,8.0,6560
3,1,68721,4.0,200000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://marvel.com/ironman3,"[{""id"": 949, ""name"": ""terrorist""}, {""id"": 5597...",en,Iron Man 3,When Tony Stark's world is torn apart by a for...,...,"[{""iso_3166_1"": ""CN"", ""name"": ""China""}, {""iso_...",2013-04-18,1215439994,130.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Unleash the power behind the armor.,Iron Man 3,6.8,8806
4,16,27205,5.0,160000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 53, ""nam...",http://inceptionmovie.warnerbros.com/,"[{""id"": 1014, ""name"": ""loss of lover""}, {""id"":...",en,Inception,"Cobb, a skilled thief who commits corporate es...",...,"[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2010-07-14,825532764,148.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Your mind is the scene of the crime.,Inception,8.1,13752


In [11]:
num_rating_df = ratings_with_name.groupby('original_title').count()[['rating']].reset_index()
num_rating_df.rename(columns={'rating': 'num_ratings'}, inplace=True)

In [12]:
num_rating_df

,original_title,num_ratings
0,(500) Days of Summer,3
1,...E tu vivrai nel terrore! L'aldilà,1
2,10 Cloverfield Lane,3
3,10 Days in a Madhouse,1
4,10 Things I Hate About You,5
...,...,...
3405,괴물,3
3406,올드보이,9
3407,인천상륙작전,2
3408,"좋은 놈, 나쁜 놈, 이상한 놈",2


In [13]:
avg_rating_df = (
    ratings_with_name
    .groupby('original_title', as_index=False)['rating']
    .mean()
    .rename(columns={'rating': 'avg_ratings'})
)

avg_rating_df


,original_title,avg_ratings
0,(500) Days of Summer,3.333333
1,...E tu vivrai nel terrore! L'aldilà,4.500000
2,10 Cloverfield Lane,3.500000
3,10 Days in a Madhouse,3.500000
4,10 Things I Hate About You,3.400000
...,...,...
3405,괴물,3.500000
3406,올드보이,3.333333
3407,인천상륙작전,4.500000
3408,"좋은 놈, 나쁜 놈, 이상한 놈",3.750000


In [14]:
popular_df = num_rating_df.merge(avg_rating_df, on='original_title')

In [15]:
popular_df

,original_title,num_ratings,avg_ratings
0,(500) Days of Summer,3,3.333333
1,...E tu vivrai nel terrore! L'aldilà,1,4.500000
2,10 Cloverfield Lane,3,3.500000
3,10 Days in a Madhouse,1,3.500000
4,10 Things I Hate About You,5,3.400000
...,...,...,...
3405,괴물,3,3.500000
3406,올드보이,9,3.333333
3407,인천상륙작전,2,4.500000
3408,"좋은 놈, 나쁜 놈, 이상한 놈",2,3.750000


In [16]:
popular_df = popular_df[popular_df['num_ratings']>=10].sort_values('avg_ratings', ascending=False).head(50)

In [17]:
popular_df = popular_df.merge(movies, on='original_title')

In [18]:
popular_df.head(1)

,original_title,num_ratings,avg_ratings,budget,genres,homepage,movie_id,keywords,original_language,overview,...,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,The Maze Runner,11,4.0,34000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 9648, ""n...",http://themazerunnermovie.com/,198663,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,"Set in a post-apocalyptic world, young Thomas ...",...,"[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2014-09-10,348319861,113.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,Run - Remember - Survive,The Maze Runner,7.0,5371


In [19]:
popular_df = popular_df[['original_title','movie_id','num_ratings','avg_ratings']]
popular_df

,original_title,movie_id,num_ratings,avg_ratings
0,The Maze Runner,198663,11,4.000000
1,Gravity,49047,11,3.954545
2,Dawn of the Planet of the Apes,119450,19,3.947368
3,Harry Potter and the Chamber of Secrets,672,11,3.909091
4,Tomorrowland,158852,10,3.900000
5,Iron Man 3,68721,12,3.875000
6,Gladiator,98,12,3.875000
7,千と千尋の神隠し,129,11,3.863636
8,Furious 7,168259,17,3.852941
9,Interstellar,157336,30,3.850000


##Collaborative Recommendation System

In [20]:
x = ratings_with_name.groupby('user_id').count()['rating'] > 0
experienced_users = x[x].index

In [21]:
filtered_rating = ratings_with_name[ratings_with_name['user_id'].isin(experienced_users)]

In [22]:
y = filtered_rating.groupby('original_title').count()['rating']>=3

famous_movies = y[y].index

In [23]:
final_ratings = filtered_rating[filtered_rating['original_title'].isin(famous_movies)]

In [24]:
pt = final_ratings.pivot_table(index='original_title', columns='user_id', values='rating')

In [25]:
pt.fillna(0, inplace=True)

In [26]:
pt

user_id,1,2,3,4,5,6,7,8,9,10,...,780,781,784,785,786,788,789,798,799,800
original_title,,,,,,,,,,,,,,,,,,,,,
(500) Days of Summer,3.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Cloverfield Lane,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Things I Hate About You,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11:14,0.0,4.0,0.0,3.0,3.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12 Angry Men,3.5,4.5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
紅番區,3.0,3.5,0.0,0.0,3.5,0.0,1.5,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
英雄,3.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,3.5,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
黃石的孩子,5.0,0.0,0.0,2.5,4.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
similarity_scores = cosine_similarity(pt)

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: divide by zero encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: overflow encountered in matmul
  ret = a @ b
/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/sklearn/utils/extmath.py:203: RuntimeWarning: invalid value encountered in matmul
  ret = a @ b


In [28]:
similarity_scores

array([[1.        , 0.70248558, 0.23543548, ..., 0.37424059, 0.74463471,
        0.51157623],
       [0.70248558, 1.        , 0.        , ..., 0.        , 0.7114094 ,
        0.25225801],
       [0.23543548, 0.        , 1.        , ..., 0.33285816, 0.        ,
        0.22016498],
       ...,
       [0.37424059, 0.        , 0.33285816, ..., 1.        , 0.        ,
        0.40246217],
       [0.74463471, 0.7114094 , 0.        , ..., 0.        , 1.        ,
        0.25225801],
       [0.51157623, 0.25225801, 0.22016498, ..., 0.40246217, 0.25225801,
        1.        ]], shape=(1585, 1585))

In [29]:
def recommend(movie_name):
    index = np.where(pt.index == movie_name)[0][0]
    similar_items = sorted(
        list(enumerate(similarity_scores[index])),
        key=lambda x: x[1],
        reverse=True
    )[0:5]  # skip the first one (it's the same movie)

    data = []
    for i, score in similar_items:
        item = []

        temp_df = movies[movies['original_title'] == pt.index[i]]
        temp_df = temp_df.drop_duplicates('original_title')

        # Movie title
        title = temp_df['original_title'].values[0]

        # Clean movie_id (remove np.int64)
        movie_id = int(temp_df['movie_id'].values[0])

        item.append(title)
        item.append(movie_id)

        data.append(item)

    return data



In [30]:
recommend('Never Let Me Go')

[['Never Let Me Go', 42188],
 ['Valkyrie', 2253],
 ['Transformers: Dark of the Moon', 38356],
 ['Over Her Dead Body', 13160],
 ['Defendor', 34769]]

In [31]:
import pickle
pickle.dump(popular_df,open('model/popular.pkl','wb'))

In [32]:
movies.drop_duplicates('original_title')

,budget,genres,homepage,movie_id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""name"": ""Adventure""}, {""id"": 14, ""...",http://disney.go.com/disneypictures/pirates/,285,"[{""id"": 270, ""name"": ""ocean""}, {""id"": 726, ""na...",en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500
2,245000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.sonypictures.com/movies/spectre/,206647,"[{""id"": 470, ""name"": ""spy""}, {""id"": 818, ""name...",en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,"[{""name"": ""Columbia Pictures"", ""id"": 5}, {""nam...","[{""iso_3166_1"": ""GB"", ""name"": ""United Kingdom""...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",http://www.thedarkknightrises.com/,49026,"[{""id"": 849, ""name"": ""dc comics""}, {""id"": 853,...",en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,"[{""name"": ""Legendary Pictures"", ""id"": 923}, {""...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106
4,260000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://movies.disney.com/john-carter,49529,"[{""id"": 818, ""name"": ""based on novel""}, {""id"":...",en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,"[{""name"": ""Walt Disney Pictures"", ""id"": 2}]","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4798,220000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 80, ""nam...",NaN,9367,"[{""id"": 5616, ""name"": ""united states\u2013mexi...",es,El Mariachi,El Mariachi just wants to play his guitar and ...,14.269792,"[{""name"": ""Columbia Pictures"", ""id"": 5}]","[{""iso_3166_1"": ""MX"", ""name"": ""Mexico""}, {""iso...",1992-09-04,2040920,81.0,"[{""iso_639_1"": ""es"", ""name"": ""Espa\u00f1ol""}]",Released,"He didn't come looking for trouble, but troubl...",El Mariachi,6.6,238
4799,9000,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 10749, ""...",NaN,72766,[],en,Newlyweds,A newlywed couple's honeymoon is upended by th...,0.642552,[],[],2011-12-26,0,85.0,[],Released,A newlywed couple's honeymoon is upended by th...,Newlyweds,5.9,5
4800,0,"[{""id"": 35, ""name"": ""Comedy""}, {""id"": 18, ""nam...",http://www.hallmarkchannel.com/signedsealeddel...,231617,"[{""id"": 248, ""name"": ""date""}, {""id"": 699, ""nam...",en,"Signed, Sealed, Delivered","""Signed, Sealed, Delivered"" introduces a dedic...",1.444476,"[{""name"": ""Front Street Pictures

In [33]:
pickle.dump(pt,open('model/pt.pkl','wb'))
pickle.dump(movies,open('model/movies.pkl','wb'))
pickle.dump(similarity_scores,open('model/similarity_scores.pkl','wb'))

In [34]:
popular_df

,original_title,movie_id,num_ratings,avg_ratings
0,The Maze Runner,198663,11,4.000000
1,Gravity,49047,11,3.954545
2,Dawn of the Planet of the Apes,119450,19,3.947368
3,Harry Potter and the Chamber of Secrets,672,11,3.909091
4,Tomorrowland,158852,10,3.900000
5,Iron Man 3,68721,12,3.875000
6,Gladiator,98,12,3.875000
7,千と千尋の神隠し,129,11,3.863636
8,Furious 7,168259,17,3.852941
9,Interstellar,157336,30,3.850000
